In [1]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row

from pyspark.ml.feature import StringIndexer

In [2]:
rdd = sc.textFile('projeto4_telecom_treino.csv')

In [3]:
rdd.take(2)

['"","state","account_length","area_code","international_plan","voice_mail_plan","number_vmail_messages","total_day_minutes","total_day_calls","total_day_charge","total_eve_minutes","total_eve_calls","total_eve_charge","total_night_minutes","total_night_calls","total_night_charge","total_intl_minutes","total_intl_calls","total_intl_charge","number_customer_service_calls","churn"',
 '"1","KS",128,"area_code_415","no","yes",25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10,3,2.7,1,"no"']

In [4]:
# QUando uisar tem que remover a primeira posicao
first = rdd.first()

In [5]:
header = first.split(',')[1:]

In [6]:
rdd2 = rdd.filter(lambda x: first not in x)
rdd2.take(2)

['"1","KS",128,"area_code_415","no","yes",25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10,3,2.7,1,"no"',
 '"2","OH",107,"area_code_415","no","yes",26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,"no"']

In [7]:
rdd3 = rdd2.map(lambda x: x.replace('"yes"', '1').replace('"no"','0'))
rdd3.take(2)

['"1","KS",128,"area_code_415",0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10,3,2.7,1,0',
 '"2","OH",107,"area_code_415",0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,0']

In [8]:
rdd4 = rdd3.map(lambda x: x.split(',')[1:len(x)])

In [9]:
rdd4.take(2)

[['"KS"',
  '128',
  '"area_code_415"',
  '0',
  '1',
  '25',
  '265.1',
  '110',
  '45.07',
  '197.4',
  '99',
  '16.78',
  '244.7',
  '91',
  '11.01',
  '10',
  '3',
  '2.7',
  '1',
  '0'],
 ['"OH"',
  '107',
  '"area_code_415"',
  '0',
  '1',
  '26',
  '161.6',
  '123',
  '27.47',
  '195.5',
  '103',
  '16.62',
  '254.4',
  '103',
  '11.45',
  '13.7',
  '3',
  '3.7',
  '1',
  '0']]

In [10]:
rdd5 = rdd4.map(lambda x: Row(state=x[0],          
                              account_length=int(x[1]),     
                              area_code=x[2],          
                              international_plan=int(x[3]),     
                              voice_mail_plan=int(x[4]),    
                              number_vmail_messages=int(x[5]),    
                              total_day_minutes=float(x[6]),   
                              total_day_calls=int(x[7]),    
                              total_day_charge=float(x[8]),  
                              total_eve_minutes=float(x[9]),   
                              total_eve_calls=int(x[10]),   
                              total_eve_charge=float(x[11]), 
                              total_night_minutes=float(x[12]),  
                              total_night_calls=int(x[13]),   
                              total_night_charge=float(x[14]), 
                              total_intl_minutes=float(x[15]),  
                              total_intl_calls=int(x[16]),   
                              total_intl_charge=float(x[17]), 
                              number_customer_service_calls=int(x[18]),  
                              churn=int(x[19])        
                             ))

In [11]:
# creating SparkSession
sps = SparkSession.builder.master('local').appName('customer_churn').config("spark.some.config.option", "some-value").getOrCreate()

In [12]:
df = sps.createDataFrame(rdd5)

In [13]:
# Convertendo categorica em numerico
indexer = StringIndexer(inputCol="area_code", outputCol="area_code_t")
df = indexer.fit(df).transform(df)

indexer2 = StringIndexer(inputCol="state", outputCol="state_t")
df = indexer2.fit(df).transform(df)

In [17]:
df.limit(5).toPandas()

,account_length,area_code,churn,international_plan,number_customer_service_calls,number_vmail_messages,state,total_day_calls,total_day_charge,total_day_minutes,...,total_eve_minutes,total_intl_calls,total_intl_charge,total_intl_minutes,total_night_calls,total_night_charge,total_night_minutes,voice_mail_plan,area_code_t,state_t
0,128,"""area_code_415""",0,0,1,25,"""KS""",110,45.07,265.1,...,197.4,3,2.70,10.0,91,11.01,244.7,1,0.0,17.0
1,107,"""area_code_415""",0,0,1,26,"""OH""",123,27.47,161.6,...,195.5,3,3.70,13.7,103,11.45,254.4,1,0.0,5.0
2,137,"""area_code_415""",0,0,0,0,"""NJ""",114,41.38,243.4,...,121.2,5,3.29,12.2,104,7.32,162.6,0,0.0,19.0
3,84,"""area_code_408""",0,1,2,0,"""OH""",71,50.90,299.4,...,61.9,7,1.78,6.6,89,8.86,196.9,0,2.0,5.0
4,75,"""area_code_415""",0,1,3,0,"""OK""",113,28.34,166.7,...,148.3,3,2.73,10.1,121,8.41,186.9,0,0.0,34.0


In [21]:
df.columns

['account_length',
 'area_code',
 'churn',
 'international_plan',
 'number_customer_service_calls',
 'number_vmail_messages',
 'state',
 'total_day_calls',
 'total_day_charge',
 'total_day_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_eve_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'total_intl_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_night_minutes',
 'voice_mail_plan',
 'area_code_t',
 'state_t']

## Describe nos totais

In [22]:
# total minutes
df.describe('total_day_minutes','total_eve_minutes','total_night_minutes','total_intl_minutes').show()

+-------+------------------+-----------------+-------------------+------------------+
|summary| total_day_minutes|total_eve_minutes|total_night_minutes|total_intl_minutes|
+-------+------------------+-----------------+-------------------+------------------+
|  count|              3333|             3333|               3333|              3333|
|   mean|179.77509750975088|200.9803480348034| 200.87203720372082|10.237293729372928|
| stddev|  54.4673892023714|  50.713844425812|  50.57384701365836|2.7918395484084138|
|    min|               0.0|              0.0|               23.2|               0.0|
|    max|             350.8|            363.7|              395.0|              20.0|
+-------+------------------+-----------------+-------------------+------------------+



In [23]:
# total calls
df.describe('total_day_calls','total_eve_calls','total_night_calls','total_intl_calls').show()

+-------+------------------+------------------+------------------+------------------+
|summary|   total_day_calls|   total_eve_calls| total_night_calls|  total_intl_calls|
+-------+------------------+------------------+------------------+------------------+
|  count|              3333|              3333|              3333|              3333|
|   mean|100.43564356435644|100.11431143114311|100.10771077107711|4.4794479447944795|
| stddev|20.069084207300907|19.922625293943103|19.568609346058555|2.4612142705460935|
|    min|                 0|                 0|                33|                 0|
|    max|               165|               170|               175|                20|
+-------+------------------+------------------+------------------+------------------+



In [24]:
# total charge
df.describe('total_day_charge','total_eve_charge','total_night_charge','total_intl_charge').show()

+-------+------------------+------------------+------------------+------------------+
|summary|  total_day_charge|  total_eve_charge|total_night_charge| total_intl_charge|
+-------+------------------+------------------+------------------+------------------+
|  count|              3333|              3333|              3333|              3333|
|   mean|30.562307230723054|17.083540354035424|  9.03932493249324| 2.764581458145822|
| stddev| 9.259434553930506|4.3106676431103415| 2.275872837660028|0.7537726126630463|
|    min|               0.0|               0.0|              1.04|               0.0|
|    max|             59.64|             30.91|             17.77|               5.4|
+-------+------------------+------------------+------------------+------------------+



## Describe nas numericas

In [25]:
df.describe('account_length','number_customer_service_calls','number_vmail_messages').show()

+-------+------------------+-----------------------------+---------------------+
|summary|    account_length|number_customer_service_calls|number_vmail_messages|
+-------+------------------+-----------------------------+---------------------+
|  count|              3333|                         3333|                 3333|
|   mean|101.06480648064806|           1.5628562856285628|    8.099009900990099|
| stddev| 39.82210592859562|           1.3154910448664763|   13.688365372038595|
|    min|                 1|                            0|                    0|
|    max|               243|                            9|                   51|
+-------+------------------+-----------------------------+---------------------+



## Count nas categoricas

Churn: yes ---> Cliente não quer mais a empresa

In [27]:
for cat_col in df.select('international_plan','voice_mail_plan','area_code','state', 'churn'):
    df.groupBy(cat_col).count().orderBy('count', ascending=0).show(5)

+------------------+-----+
|international_plan|count|
+------------------+-----+
|                 0| 3010|
|                 1|  323|
+------------------+-----+

+---------------+-----+
|voice_mail_plan|count|
+---------------+-----+
|              0| 2411|
|              1|  922|
+---------------+-----+

+---------------+-----+
|      area_code|count|
+---------------+-----+
|"area_code_415"| 1655|
|"area_code_510"|  840|
|"area_code_408"|  838|
+---------------+-----+

+-----+-----+
|state|count|
+-----+-----+
| "WV"|  106|
| "MN"|   84|
| "NY"|   83|
| "AL"|   80|
| "OR"|   78|
+-----+-----+
only showing top 5 rows

+-----+-----+
|churn|count|
+-----+-----+
|    0| 2850|
|    1|  483|
+-----+-----+



In [ ]:
tps = [
    ('account_length', 'num'),
    ('churn', 'cat'),
    ('international_plan', 'cat'),
    ('num', 'bigint'),
    ('number_vmail_messages', 'num'),
    ('total_day_calls', 'num'),
    ('total_day_charge', 'num'),
    ('total_day_minutes', 'num'),
    ('total_eve_calls', 'num'),
    ('total_eve_charge', 'num'),
    ('total_eve_minutes', 'num'),
    ('total_intl_calls', 'num'),
    ('total_intl_charge', 'num'),
    ('total_intl_minutes', 'num'),
    ('total_night_calls', 'num'),
    ('total_night_charge', 'num'),
    ('total_night_minutes', 'num'),
    ('voice_mail_plan', 'cat'),
    ('area_code_t', 'cat'),
    ('state_t', 'cat')
]

In [29]:
df.show()

+--------------+---------------+-----+------------------+-----------------------------+---------------------+-----+---------------+----------------+-----------------+---------------+----------------+-----------------+----------------+-----------------+------------------+-----------------+------------------+-------------------+---------------+-----------+-------+
|account_length|      area_code|churn|international_plan|number_customer_service_calls|number_vmail_messages|state|total_day_calls|total_day_charge|total_day_minutes|total_eve_calls|total_eve_charge|total_eve_minutes|total_intl_calls|total_intl_charge|total_intl_minutes|total_night_calls|total_night_charge|total_night_minutes|voice_mail_plan|area_code_t|state_t|
+--------------+---------------+-----+------------------+-----------------------------+---------------------+-----+---------------+----------------+-----------------+---------------+----------------+-----------------+----------------+-----------------+------------------